In [1]:
import importlib
import plot_opto
import classify_opto
import preprocess_opto
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
importlib.reload(plot_opto)
importlib.reload(classify_opto)
importlib.reload(preprocess_opto)
# mouse and date
mouse = 'NN13'
date = '210811'
# create folders to save files
paths = preprocess_opto.create_folders(mouse, date)
# process and plot behavior
behavior = preprocess_opto.process_behavior(paths)
# save masks so can run in matlab to process other planes
preprocess_opto.cell_masks(paths, 0)

In [5]:
importlib.reload(plot_opto)
importlib.reload(classify_opto)
importlib.reload(preprocess_opto)
# grab activity 
deconvolved = preprocess_opto.process_activity(paths, 'spks', 3, 0)
# normalize activity
norm_deconvolved = preprocess_opto.normalize_deconvolved(deconvolved, behavior, paths, 0)
# gassuain filter acitivity
norm_moving_deconvolved_filtered = preprocess_opto.difference_gaussian_filter(norm_deconvolved, 4, behavior, paths, 0)
# make trial averaged traces and basline subtract
mean_cs_1_responses_df = preprocess_opto.normalized_trial_averaged(norm_deconvolved, behavior, 'cs_1')
mean_cs_2_responses_df = preprocess_opto.normalized_trial_averaged(norm_deconvolved, behavior, 'cs_2')
mean_cs_1_opto_responses_df = preprocess_opto.normalized_trial_averaged(norm_deconvolved, behavior, 'cs_1_opto')
mean_cs_2_opto_responses_df = preprocess_opto.normalized_trial_averaged(norm_deconvolved, behavior, 'cs_2_opto')
# get sig cells
[cs_1_poscells, cs_1_negcells] = preprocess_opto.sig_test(norm_deconvolved, behavior, 'cs_1')
[cs_2_poscells, cs_2_negcells] = preprocess_opto.sig_test(norm_deconvolved, behavior, 'cs_2')
[both_poscells, both_sigcells] = preprocess_opto.combine_sig(cs_1_poscells, cs_1_negcells, cs_2_poscells, cs_2_negcells)
# get idx of top cell differences
idx = preprocess_opto.get_index(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, cs_1_poscells, cs_2_poscells, both_poscells, both_sigcells, paths, 0)
# plot heatmap of top cells
plot_opto.sorted_map(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, mean_cs_1_opto_responses_df, mean_cs_2_opto_responses_df, idx['cs_1'], idx['cs_2'], 800, paths)

In [ ]:
importlib.reload(plot_opto)
importlib.reload(classify_opto)
importlib.reload(preprocess_opto)
# get prior for synchronous cue activity
prior = classify_opto.prior(norm_moving_deconvolved_filtered, idx['cs_1'], idx['cs_2'], behavior, [])
# logistic regression 
y_pred = classify_opto.log_regression(behavior, norm_deconvolved, norm_moving_deconvolved_filtered, both_poscells, prior)
# process classified output
y_pred = classify_opto.process_classified(y_pred, prior, paths, 1)
# get reactivation times and save
preprocess_opto.get_reactivation_times(y_pred, behavior, 0.5, paths, 0)

In [ ]:
importlib.reload(plot_opto)
importlib.reload(classify_opto)
importlib.reload(preprocess_opto)
# plot mean reactivation probability after cues
plot_opto.mean_reactivation(y_pred, behavior, paths, [], [])
# plot all reactivations over time
plot_opto.all_reactivation(y_pred, behavior, paths, [], [])
# plot physical evoked reactivations
plot_opto.reactivation_physical(y_pred, behavior, paths, [], [])
# plot effects of opto following normal trial
plot_opto.trial_effect(y_pred, behavior, paths, [], [])
# activity control during iti
plot_opto.activity_control(norm_deconvolved, behavior, idx, paths, [], [])
# plot learning
plot_opto.activity_across_trials(norm_deconvolved, behavior, y_pred, idx, paths, [], [])
# plot reactivation raster
plot_opto.reactivation_raster(behavior, norm_deconvolved, y_pred, idx['cs_1_df'], idx['cs_2_df'], idx['both'], paths, [2])

In [ ]:
importlib.reload(plot_opto)
importlib.reload(classify_opto)
importlib.reload(preprocess_opto)
plot_opto.sample_reactivation_raster(behavior, norm_deconvolved, y_pred, idx['cs_1_df'], idx['cs_2_df'], idx['both'], paths, 88000, 89000)